In [1]:
%pip install openai==0.28
%pip install pytesseract pillow gradio


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
from dotenv import load_dotenv, find_dotenv
import gradio as gr


load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']

def extract_text_from_image(image_path):
    try:

        img = Image.open(image_path)
        img = img.convert('L') 
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2)  
        img = img.filter(ImageFilter.SHARPEN)

        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        print(f"Error: {e}")
        return ""

def identify_marked_fields_with_gpt(extracted_text):
    messages = [
        {"role": "system", "content": "You are a kind helpful assistant."},
        {"role": "user", "content": f"Here is the extracted textand from an image: \n\n{extracted_text}\n\nIdentify and list all marked fields accurately.  for every marked field you find that corresponds to a diagnosis give the CIE-10 code"},
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4o",  # Use the appropriate model name
        messages=messages
    )

    reply = response.choices[0].message['content']
    return reply


def process_images(image_paths):
    all_marked_fields = {}
    
    for image_path in image_paths:
        extracted_text = extract_text_from_image(image_path)
        marked_fields = identify_marked_fields_with_gpt(extracted_text)
        
        all_marked_fields[os.path.basename(image_path)] = marked_fields
    
    return all_marked_fields


image_paths = ["./app/Formularios/diagnostico_2.png",
"./app/Formularios/diagnostico_1.png"
]

results = process_images(image_paths)

for image_name, fields in results.items():
    print(f"Marked fields in {image_name}:")
    print(fields)
    print()


interface = gr.Interface(
    fn=process_image,
    inputs=gr.inputs.Image(type="pil"),
    outputs="text",
    title="Marked Fields Extractor",
    description="Upload an image to extract and identify marked fields using OCR and GPT-4."
)

interface.launch()

Marked fields in diagnostico_2.png:
Based on the extracted text provided, there are several fields that are marked and intended to be filled out in the document "Formulario de diagnóstico médico para jugadores con discapacidad física." The marked fields are as follows:

1. **Apellidos (Last Name):** GONZALEZ GONZALEZ
2. **Nombre (First Name):** JOSE
3. **Género (Gender):** 
   - [ ] Fem
   - [x] Masc (Male)
4. **Fecha de nacimiento (Date of Birth):** 01/05/1995 (dd/mm/yyyy)
5. **Club:** CLUB RAYO VALLECANO
6. **Licencia (License):** CDG4E6

There were no specific diagnosis-related fields filled out in the text provided. Consequently, no CIE-10 (ICD-10) codes can be provided based on the information given.

If you have more information or specific diagnoses listed in the form that you'd like help with identifying corresponding ICD-10 codes, please provide those details.

Marked fields in diagnostico_1.png:
The text provided contains fields that could be marked to indicate relevant diagn

NameError: name 'process_image' is not defined